In [ ]:
import os
import shutil
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/PQHIDE')

In [ ]:
import hashlib

import numpy as np

from utils import gaussjordan

from LinearCode import LinearCode



class QC_MDPC(LinearCode):
    """
    Quasi-Cyclic LDPC code representation (extends LinearCode)

    ...

    Methods
    -------
    from_params(n, p, w)
        Init QC-LDPC by length, circulant size and code weight

    _get_circulant_block(polynom)
        Get circulant (p, p) for given vector of size p

    """

    def __init__(self, G, H):
        super().__init__(G, H)

    @classmethod
    def from_params(cls, n, p, w):
        assert n % p == 0, "p must be delimeter of n"

        n0 = n // p
        assert w > 2*n0, "not enough code weight"

        fine = False

        while not fine:
            blocks = []
            inverse_block = None
            inverse_block_position = None

            vector = [1 for _ in range(w)] + [0 for _  in range(n - w)]
            vector = np.array(vector, dtype=int)
            np.random.shuffle(vector)

            for i in range(n0):
                circ = vector[i*p:(i+1)*p]

                if sum(circ) < 2:
                    inverse_block = None
                    break

                block = QC_MDPC._get_circulant_block(circ)
                blocks.append(block)

                A, P = gaussjordan(block, True)
                A = np.array(A, dtype=int)
                P = np.array(P, dtype=int)

                if (A == np.eye(p, dtype=int)).all():
                    inverse_block_position = i
                    inverse_block = P

            # continue only if inverse circulant found
            fine = True if inverse_block is not None else False

        # put inverse block on last position
        blocks[inverse_block_position], blocks[n0-1] = blocks[n0-1], blocks[inverse_block_position]
        H = np.concatenate(blocks, axis=1)

        for i in range(n0):
            blocks[i] = blocks[i] @ inverse_block % 2
            blocks[i] = blocks[i].T

        Ht = np.concatenate(blocks[:n0-1], axis=0)
        G = np.concatenate((np.eye(Ht.shape[0], dtype=int), Ht), axis=1)

        assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"

        return cls(G, H)

    @staticmethod
    def _get_circulant_block(polynom):
        N = len(polynom)
        block = np.empty((N, N), dtype=int)

        for i in range(N):
            block[i] = np.roll(polynom, i)

        return block



# EXAMPLE USAGE:

n = 8
p = 4
w = 5
errors_num = 2
qc_mdpc = QC_MDPC.from_params(n, p, w)

word = np.random.randint(2, size=qc_mdpc.getG().shape[0])
print(word)

encoded = qc_mdpc.encode(word)
print(encoded)
print(len(encoded))

# error vector size n with t or less errors
e = [1 for _ in range(errors_num)] + [0 for _  in range(n - errors_num)]
e = np.array(e, dtype=int)
np.random.shuffle(e)
print(e)
print(len(e))

corrupted = (encoded + e) % 2
print(corrupted)
print(len(corrupted))

decoded = qc_mdpc.decode(np.copy(corrupted))
decoded = qc_mdpc.get_message(decoded)
print(decoded)
print(len(decoded))

try:
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be reforwarded")

In [5]:
os.chdir('/content/gdrive/MyDrive/PQHIDE/Species')

In [6]:
folder_path = '/content/gdrive/MyDrive/PQHIDE/Species/file.txt'

folder_names = []

with open(folder_path, 'r') as f:
  for line in f:
    folder_names.append(line.strip()) # strip() removes potential newline characters


# Create the empty folders
for folder_name in folder_names:
  if not os.path.exists(folder_name):
      os.makedirs(folder_name)

In [ ]:
count = 0
for folder_name in folder_names:
    if(os.path.isdir(os.path.join(folder_name))):
        count = count + 1

print(count)

In [ ]:
ls

In [9]:
value = 0
dico = {}
for filename in os.listdir('/content/gdrive/MyDrive/PQHIDE/Species'):
        if(os.path.isdir(filename)):
          dico[value]  = filename
          value = value + 1

In [ ]:
 for i in dico.items():
    print(i)

In [ ]:
y = corrupted
print(y)
len(y)

In [ ]:
lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/MyDrive/PQHIDE/Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

In [ ]:
lis

In [14]:
def hash_directory(path):
    digest = hashlib.sha256()

    for root, dirs, files in os.walk(path):
        for names in files:
            file_path = os.path.join(root, names)

            # Hash the path and add to the digest to account for empty files/directories
            digest.update(hashlib.sha1(file_path[len(path):].encode()).digest())

            # Per @pt12lol - if the goal is uniqueness over repeatability, this is an alternative method using 'hash'
            # digest.update(str(hash(file_path[len(path):])).encode())

            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f_obj:
                    while True:
                        buf = f_obj.read(1024 * 1024)
                        if not buf:
                            break
                        digest.update(buf)

    return digest.hexdigest()

In [ ]:
list_Hash1 = []
for i in range(len(y)):
    list_Hash1.append('')
list_Hash1

In [ ]:
for i in range(len(y)):
    path = '/content/gdrive/MyDrive/PQHIDE/Species/' + dico[i]
    list_Hash1[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash1[i])

In [ ]:
y

In [18]:
liste = ['Balistidae', 'Belonidae', 'Enoploteuthidae', 'Cyprinidae', 'Nephropidae', 'Axiidae', 'Serranidae', 'Cardiidae', 'Scombridae', 'Sparidae', 'Acanthuridae', 'Limidae', 'Sergestidae', 'Acipenseridae', 'Acropomatidae', 'Holothuriidae', 'Squalidae', 'Octopodidae', 'Pectinidae', 'Serranidae']

In [ ]:
len(liste)

In [20]:
i = 0
while i < len(y):
    number = np.random.randint(0,20)
    file = '/'+liste[number]+'.txt'
    if(y[i] == 1):
        my_path = '/content/gdrive/MyDrive/PQHIDE/Species/' + dico[i]
        os.chdir(my_path)
        if not os.path.exists(my_path + file):
            with open(my_path + file,'w') as fp:
                pass
        else:
            descriptor = open(my_path + file,'a')
            descriptor.write("Species")
            descriptor.close()
    i = i + 1

In [ ]:
lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/MyDrive/PQHIDE/Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

In [ ]:
list_Hash2 = []
for i in range(len(y)):
    list_Hash2.append('')
list_Hash2

In [ ]:
for i in range(len(y)):
    path = '/content/gdrive/MyDrive/PQHIDE/Species/' + dico[i]
    list_Hash2[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash2[i])

In [24]:
msg_secret = []
for i in range(len(y)):
    if(list_Hash1[i] != list_Hash2[i]):
        msg_secret.append(1)
    else:
        msg_secret.append(0)

In [ ]:
np.array(msg_secret)

In [ ]:
y

In [ ]:
len(msg_secret)

In [ ]:
decoded = qc_mdpc.decode(np.copy(corrupted))
decoded = qc_mdpc.get_message(decoded)
print(decoded)
print(len(decoded))

In [ ]:
word

In [ ]:
try:
    assert (msg_secret == y).all()
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be reforwarded")
else:
    print("The secret message has been correctly forwarded")

# #Code script for time evaluation

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/PQHIDE')
import time
import os
import shutil
import numpy as np
from utils import gaussjordan
from LinearCode import LinearCode
import hashlib
from collections import defaultdict


class QC_MDPC(LinearCode):
    """
    Quasi-Cyclic LDPC code representation (extends LinearCode)

    ...

    Methods
    -------
    from_params(n, p, w)
        Init QC-LDPC by length, circulant size and code weight

    _get_circulant_block(polynom)
        Get circulant (p, p) for given vector of size p

    """

    def __init__(self, G, H):
        super().__init__(G, H)

    @classmethod
    def from_params(cls, n, p, w):
        assert n % p == 0, "p must be delimeter of n"

        n0 = n // p
        assert w > 2*n0, "not enough code weight"

        fine = False

        while not fine:
            blocks = []
            inverse_block = None
            inverse_block_position = None

            vector = [1 for _ in range(w)] + [0 for _  in range(n - w)]
            vector = np.array(vector, dtype=int)
            np.random.shuffle(vector)

            for i in range(n0):
                circ = vector[i*p:(i+1)*p]

                if sum(circ) < 2:
                    inverse_block = None
                    break

                block = QC_MDPC._get_circulant_block(circ)
                blocks.append(block)

                A, P = gaussjordan(block, True)
                A = np.array(A, dtype=int)
                P = np.array(P, dtype=int)

                if (A == np.eye(p, dtype=int)).all():
                    inverse_block_position = i
                    inverse_block = P

            # continue only if inverse circulant found
            fine = True if inverse_block is not None else False

        # put inverse block on last position
        blocks[inverse_block_position], blocks[n0-1] = blocks[n0-1], blocks[inverse_block_position]
        H = np.concatenate(blocks, axis=1)

        for i in range(n0):
            blocks[i] = blocks[i] @ inverse_block % 2
            blocks[i] = blocks[i].T

        Ht = np.concatenate(blocks[:n0-1], axis=0)
        G = np.concatenate((np.eye(Ht.shape[0], dtype=int), Ht), axis=1)

        assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"

        return cls(G, H)

    @staticmethod
    def _get_circulant_block(polynom):
        N = len(polynom)
        block = np.empty((N, N), dtype=int)

        for i in range(N):
            block[i] = np.roll(polynom, i)

        return block



# EXAMPLE USAGE:

n = 8
p = 4
w = 5
errors_num = 2

qc_mdpc = QC_MDPC.from_params(n, p, w)

start = time.time()

word = np.random.randint(2, size=qc_mdpc.getG().shape[0])
print(word)

encoded = qc_mdpc.encode(word)
print(encoded)
print(len(encoded))

# error vector size n with t or less errors
e = [1 for _ in range(errors_num)] + [0 for _  in range(n - errors_num)]
e = np.array(e, dtype=int)
np.random.shuffle(e)
print(e)
print(len(e))

corrupted = (encoded + e) % 2
print(corrupted)
print(len(corrupted))


value = 0
dico = {}
for filename in os.listdir('/content/gdrive/MyDrive/PQHIDE/Species'):
        #if(os.path.isdir(filename)):
          dico[value]  = filename
          value = value + 1

y = corrupted

list_Hash1 = []
for i in range(len(y)):
    list_Hash1.append('')


for i in range(len(y)):
    path = '/content/gdrive/MyDrive/PQHIDE/Species/' + dico[i]
    list_Hash1[i] = hash_directory(path)


liste = ['Balistidae', 'Belonidae', 'Enoploteuthidae', 'Cyprinidae', 'Nephropidae', 'Axiidae', 'Serranidae', 'Cardiidae', 'Scombridae', 'Sparidae', 'Acanthuridae', 'Limidae', 'Sergestidae', 'Acipenseridae', 'Acropomatidae', 'Holothuriidae', 'Squalidae', 'Octopodidae', 'Pectinidae', 'Serranidae']

i = 0
while i < len(y):
    number = np.random.randint(0,20)
    file = '/'+liste[number]+'.txt'
    if(y[i] == 1):
        my_path = '/content/gdrive/MyDrive/PQHIDE/Species/' + dico[i]
        os.chdir(my_path)
        if not os.path.exists(my_path + file):
            with open(my_path + file,'w') as fp:
                pass
        else:
            descriptor = open(my_path + file,'a')
            descriptor.write("Species")
            descriptor.close()
    i = i + 1



list_Hash2 = []
for i in range(len(y)):
    list_Hash2.append('')


for i in range(len(y)):
    path = '/content/gdrive/MyDrive/PQHIDE/Species/' + dico[i]
    list_Hash2[i] = hash_directory(path)


msg_secret = []
for i in range(len(y)):
    if(list_Hash1[i] != list_Hash2[i]):
        msg_secret.append(1)
    else:
        msg_secret.append(0)


decoded = qc_mdpc.decode(np.copy(corrupted))
decoded = qc_mdpc.get_message(decoded)
print(decoded)
print(len(decoded))

end = time.time()
print('time = ',end - start)

try:
    assert (msg_secret == y).all()
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be reforwarded")
else:
    print("The secret message has been correctly forwarded")